In [1]:
import librosa, librosa.display
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy
import math
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

In [6]:
#store the dataframe with MFCC coefficients and remove those with voice
df = pd.read_csv('MLNS_with_mfcc_stats_800_cutoff_05272024.csv')
no_voice_list = pd.read_csv('no_voice_files.csv', header=None)
filename_list = [name.split("\\")[-1] for name in no_voice_list[0]]
df2=df[0:0]
for name in filename_list:
    df2=pd.concat([df2, df.loc[df['cat_num']==int(name[:-4])]], ignore_index=True)
df=df2

/var/folders/5z/7fkzpcz93gz6mqgh9vjgfymm0000gn/T/ipykernel_94180/1410855225.py:2: DtypeWarning: Columns (26,32,45) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('MLNS_with_mfcc_stats_800_cutoff_05272024.csv')


In [11]:
df=df.drop(['Unnamed: 0'],axis=1)

In [14]:
df['critter_name'].value_counts()

critter_name
cricket        4068
kaydid         1992
cicada           60
grasshopper       1
Name: count, dtype: int64

In [15]:
df=df.drop(df.loc[df['critter_name']=='grasshopper'].index[0])
df = df.loc[df['mfcc_39_var'] > -10000000]
df['critter_name'].value_counts()

critter_name
cricket    3949
kaydid     1895
cicada       55
Name: count, dtype: int64

In [78]:
df.to_csv("MLNS_test.csv", index=False)

In [77]:
df.head(10)

,cat_num,format,common_name,fam_or_subfam,genus,scientific_name,background_species,recordist,date,year,...,mfcc_36_var,mfcc_37_avg,mfcc_37_var,mfcc_38_avg,mfcc_38_var,mfcc_39_avg,mfcc_39_var,main_freq,range,max_mean
0,103160,Audio,NaN,Cicadidae,Fidicina,Fidicina mannifera,NaN,Matthew D. Medler,6/27/1999,1999.0,...,11.626591,0.530423,13.374516,-2.264081,12.836982,2.457076,14.518044,10680.468750,1.060190,3273.046875
1,103337,Audio,NaN,Cicadidae,Cicadidae,Cicadidae,NaN,Matthew D. Medler,3/17/2003,2003.0,...,115.646500,-4.276080,40.552258,5.668609,81.285050,8.444180,73.539220,2411.718750,1.045662,1604.223633
2,106777,Audio,Pharaoh Cicada,Cicadidae,Magicicada,Magicicada septendecim,NaN,Randolph Little,6/18/1996,1996.0,...,9.788895,2.425860,6.631615,-3.152511,8.163149,2.040744,7.501962,1270.458984,2.205479,398.364258
4,106781,Audio,Dwarf Periodical Cicada,Cicadidae,Magicicada,Magicicada cassini,NaN,Randolph Little,6/21/1996,1996.0,...,10.601876,1.280277,9.638605,-1.152836,9.387085,1.588869,9.038735,4263.574219,1.231806,3477.612305
11,107222,Audio,narrow-winged tree cricket,Oecanthinae,Oecanthus,Oecanthus niveus,NaN,Wil Hershberger,11/3/1999,1999.0,...,9.112074,1.997714,11.733382,-0.620154,10.982013,0.867990,10.133170,1884.155273,1.203704,1055.126953
12,107224,Audio,narrow-winged tree cricket,Oecanthinae,Oecanthus,Oecanthus niveus,NaN,Wil Hershberger,11/3/1999,1999.0,...,9.301886,7.672295,17.968626,-1.437193,6.881306,-4.321786,16.575102,1894.921875,1.120482,1098.193359
14,107458,Audio,Black-legged Meadow Katydid,Conocephalinae,Orchelimum,Orchelimum nigripes,NaN,Wil Hershberger,8/21/2000,2000.0,...,10.405321,1.332214,9.954836,-1.356493,10.848813,1.105920,9.553658,10669.702148,1.208183,5297.167969
20,107470,Audio,stripe-faced meadow katydid,Conocephalinae,Orchelimum,Orchelimum concinnum,NaN,Wil Hershberger,9/2/2000,2000.0,...,5.900653,0.925960,4.595885,0.171828,3.980715,-0.164493,4.507919,11025.000000,1.005888,9087.011719
23,107481,Audio,Handsome Meadow Katydid,Conocephalinae,Orchelimum,Orchelimum pulchellum,NaN,Wil Hershberger,9/10/2000,2000.0,...,10.611208,2.274681,9.830249,-4.041643,11.117353,3.416365,10.178897,10077.539062,1.197619,8311.816406
24,107492,Audio,rattler round-winged katydid,Phaneropterinae,Amblycorypha,Amblycorypha rotundifolia,Anthus rubescens -- American Pipit,Wil Hershberger,10/4/2000,2000.0,...,8.136271,3.613848,7.343521,-1.500757,6.973211,2.569118,8.111092,10562.036133,1.260377,9420.776367


In [18]:
df2 = pd.read_csv('MLNS_with_main_freq_and_range_05272024.csv')

In [71]:
float((df2.loc[df2['cat_num']==103160, 'main_freq']))

/var/folders/5z/7fkzpcz93gz6mqgh9vjgfymm0000gn/T/ipykernel_94180/4197938965.py:1: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float((df2.loc[df2['cat_num']==103160, 'main_freq']))


10680.46875

In [76]:
for num in df['cat_num']:
    df.at[df.loc[df['cat_num']==num].index[0], 'max_mean'] = float((df2.loc[df2['cat_num']==num, 'max_mean']))

/var/folders/5z/7fkzpcz93gz6mqgh9vjgfymm0000gn/T/ipykernel_94180/2904777395.py:2: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  df.at[df.loc[df['cat_num']==num].index[0], 'max_mean'] = float((df2.loc[df2['cat_num']==num, 'max_mean']))
/var/folders/5z/7fkzpcz93gz6mqgh9vjgfymm0000gn/T/ipykernel_94180/2904777395.py:2: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  df.at[df.loc[df['cat_num']==num].index[0], 'max_mean'] = float((df2.loc[df2['cat_num']==num, 'max_mean']))


In [165]:
#Change critter name to a number
df.loc[df['critter_name']=='cricket', 'critter_num'] = 0
df.loc[df['critter_name']=='kaydid', 'critter_num'] = 1
df.loc[df['critter_name']=='cicada', 'critter_num'] = 2

In [192]:
df['critter_num'].value_counts()

critter_num
0.0    3949
1.0    1895
2.0      55
Name: count, dtype: int64

In [193]:
df['fam_or_subfam'].value_counts()

fam_or_subfam
Gryllinae           1419
Conocephalinae       847
Oecanthinae          745
Phaneropterinae      624
Trigonidiinae        553
Nemobiinae           386
Hapithinae           380
Mogoplistinae        372
Tettigoniinae        285
Pseudophyllinae      139
Cicadidae             55
Gryllotalpidae        41
Eneopterinae          31
Phalangopsidae        12
Listroscelidinae      10
Name: count, dtype: int64

In [72]:
#For later use, store the dataframe with only crickets and kaydids
df_bin = df.drop(df[df.critter_num == 2].index)

In [81]:
#Returns features truncated to indicated depth
def truncate_mfcc(df, avg_depth=20, var_depth=10):
    df_temp=df['mfcc_'+str(0)+'_avg']
    df_temp=df_temp.rename('blah')
    for n in range(0,avg_depth):
        df_temp=pd.concat([df_temp, df['mfcc_'+str(n)+'_avg']], axis=1)
    for n in range(0,var_depth):
        df_temp=pd.concat([df_temp, df['mfcc_'+str(n)+'_var']], axis=1)        
    return df_temp.drop(columns=['blah'])

In [61]:
#LDA for all three critter names with 80 mfcc features
X=truncate_mfcc(df, avg_depth=40, var_depth=40)
Y = df['critter_num']

In [62]:
X_train, X_test, y_train, y_test = train_test_split(X.copy(), Y,
                                                       shuffle=True,
                                                       random_state=413,
                                                       test_size=.2,
                                                       stratify=Y)

In [64]:
## Make LDA object
LDA = LinearDiscriminantAnalysis()

## Fit the model
LDA.fit(X_train, y_train)

LinearDiscriminantAnalysis()

In [66]:
conf_mat = confusion_matrix(y_test, LDA.predict(X_test))

In [67]:
pd.DataFrame(conf_mat,
                 columns = ['Predicted 0', 'Predicted 1', 'Predicted 2'],
                 index = ['Actual 0', 'Actual 1', 'Actual 2'])

,Predicted 0,Predicted 1,Predicted 2
Actual 0,735,57,3
Actual 1,95,287,3
Actual 2,6,1,5


In [52]:
#LDA for two critter names with 80 mfcc features
X=truncate_mfcc(df_bin, avg_depth=40, var_depth=40)
Y = df_bin['critter_num']

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X.copy(), Y,
                                                       shuffle=True,
                                                       random_state=413,
                                                       test_size=.2,
                                                       stratify=Y)

In [54]:
## Make LDA object
LDA = LinearDiscriminantAnalysis()

## Fit the model
LDA.fit(X_train, y_train)

LinearDiscriminantAnalysis()

In [55]:
conf_mat = confusion_matrix(y_train, LDA.predict(X_train))

In [56]:
pd.DataFrame(conf_mat,
                 columns = ['Predicted 0', 'Predicted 1'],
                 index = ['Actual 0', 'Actual 1'])

,Predicted 0,Predicted 1
Actual 0,2983,197
Actual 1,326,1215


In [57]:
conf_mat = confusion_matrix(y_test, LDA.predict(X_test))

In [58]:
pd.DataFrame(conf_mat,
                 columns = ['Predicted 0', 'Predicted 1'],
                 index = ['Actual 0', 'Actual 1'])

,Predicted 0,Predicted 1
Actual 0,737,59
Actual 1,96,289


In [116]:
#KNN
X=truncate_mfcc(df, avg_depth=10, var_depth=10)
Y = df['critter_num']

In [117]:
X_train, X_test, y_train, y_test = train_test_split(X.copy(), Y,
                                                       shuffle=True,
                                                       random_state=17,
                                                       test_size=.2,
                                                       stratify=Y)

In [154]:
## make the model object
knn_pipe = Pipeline([('scale', StandardScaler()), ('knn', KNeighborsClassifier(5))])

## "fit" the model object
knn_pipe.fit(X_train,
           y_train)

## predict on the training set
pred = knn_pipe.predict(X_test)

In [79]:
## We define it by hand here
## but you can also use accuracy_score from sklearn
## https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html
def accuracy(true, predicted):
    return np.sum(true==predicted)/len(predicted)

from sklearn.metrics import accuracy_score

In [83]:
for depth in range(1,40):
    for k in range(5,6):
        X=truncate_mfcc(df, avg_depth=depth, var_depth=depth)
        Y = df['critter_name']
      #  Y = df['fam_or_subfam']
        X_train, X_test, y_train, y_test = train_test_split(X.copy(), Y, shuffle=True, random_state=17, test_size=.2, stratify=Y)
        knn_pipe = Pipeline([('scale', StandardScaler()), ('knn', KNeighborsClassifier(k))])
        knn_pipe.fit(X_train, y_train)
        pred = knn_pipe.predict(X_test)
        score = accuracy_score(y_test,pred)
        print(f"Test Accuracy when mfcc depth = {depth} and k = {k}: {score}")

Test Accuracy when mfcc depth = 1 and k = 5: 0.6237288135593221
Test Accuracy when mfcc depth = 2 and k = 5: 0.6915254237288135
Test Accuracy when mfcc depth = 3 and k = 5: 0.7652542372881356
Test Accuracy when mfcc depth = 4 and k = 5: 0.8228813559322034
Test Accuracy when mfcc depth = 5 and k = 5: 0.8449152542372881
Test Accuracy when mfcc depth = 6 and k = 5: 0.8559322033898306
Test Accuracy when mfcc depth = 7 and k = 5: 0.864406779661017
Test Accuracy when mfcc depth = 8 and k = 5: 0.8711864406779661
Test Accuracy when mfcc depth = 9 and k = 5: 0.8711864406779661
Test Accuracy when mfcc depth = 10 and k = 5: 0.8779661016949153
Test Accuracy when mfcc depth = 11 and k = 5: 0.8728813559322034
Test Accuracy when mfcc depth = 12 and k = 5: 0.8754237288135593
Test Accuracy when mfcc depth = 13 and k = 5: 0.8813559322033898
Test Accuracy when mfcc depth = 14 and k = 5: 0.8838983050847458
Test Accuracy when mfcc depth = 15 and k = 5: 0.8847457627118644
Test Accuracy when mfcc depth = 16 